In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data=pd.read_csv('/content/data_final preprocessed_cdms.csv')
data

,cat__Varicella_1,cat__Varicella_2,cat__Mono or polysymptomatic_2,cat__Oligoclonal bands_0,cat__Oligoclonal bands_1,cat__Oligoclonal bands_2,cat__initial EDSS_1.0,cat__initial EDSS_2.0,cat__initial symptom_1.0,cat__initial symptom_2.0,...,cat__initial symptom_nan,cat__ final EDSS_1.0,cat__ final EDSS_2.0,remainder__Gender,remainder__LLSSEP,remainder__VEP,remainder__BAEP,remainder__Periventricular MRI,remainder__Infratentorial MRI,remainder__group
0,1,0,0,1,0,0,1,0,0,1,...,0,1,0,1,1,0,0,0,0,1
1,0,1,1,0,1,0,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1
2,1,0,0,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
3,1,0,1,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4,1,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
268,0,0,1,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0
269,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
270,0,0,1,0,1,0,1,0,0,0,...,0,1,0,0,1,1,0,1,1,0


In [3]:
x=data.drop('remainder__group',axis=1)
y=data['remainder__group']

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)

In [5]:
from imblearn.over_sampling import BorderlineSMOTE
Borderline = BorderlineSMOTE(random_state=42)

x_train_res, y_train_res = Borderline.fit_resample(x_train, y_train)

In [6]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.0 MB/s eta 0:00:00


In [7]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define search space
param_space = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, None],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_leaf': [1, 2, 7, 12],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 200, 300, 400]
}


opt = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    search_spaces=param_space,
    cv=5,          # cross-validation
    n_jobs=-1,     # all cores
    n_iter=50,     # number of iterations for Bayesian optimization (recommended)
    random_state=42,
    verbose=0,
    refit=True
)

# Fit on training data
opt.fit(x_train_res, y_train_res)

# Predictions
y_pred = opt.predict(x_test)
y_prob = opt.predict_proba(x_test)[:, 1]


In [8]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score,accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("PR AUC (average precision):", average_precision_score(y_test, y_prob))

Accuracy: 0.9272727272727272
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        30
           1       0.89      0.96      0.92        25

    accuracy                           0.93        55
   macro avg       0.93      0.93      0.93        55
weighted avg       0.93      0.93      0.93        55

Confusion matrix:
 [[27  3]
 [ 1 24]]
ROC AUC: 0.952
PR AUC (average precision): 0.9501329045567435


In [9]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from skopt import BayesSearchCV
from skopt.space import Categorical, Real # Import Real for continuous parameters


# Define models and hyperparameters
models_and_params = {
    "KNN": (
        Pipeline([("scaler", StandardScaler()), ("model", KNeighborsClassifier())]),
        {
            "model__n_neighbors": [3, 5, 7],
            "model__weights": ["uniform", "distance"],
            "model__leaf_size": [20, 30]
        }
    ),
    "SVM": (
        Pipeline([("scaler", StandardScaler()), ("model", SVC(probability=True))]),
        {
            "model__C": [0.1, 1, 10],
            "model__kernel": ["linear", "rbf"],
            "model__gamma": ["scale", "auto"]
        }
    ),
    "Gaussian Process": (
        GaussianProcessClassifier(random_state=42, kernel=RBF()), # Provide a base RBF kernel
        {
            "kernel__length_scale": Real(0.1, 5.0, 'log-uniform'), # Tune length_scale as a continuous parameter
            "n_restarts_optimizer": [0, 5],
            "max_iter_predict": [100]
        }
    ),
    "Decision Tree": (
        DecisionTreeClassifier(random_state=42),
        {
            "criterion": ["gini", "entropy"],
            "max_depth": [3, 5, 7, None],
            "min_samples_leaf": [1, 2, 4]
        }
    ),
    "AdaBoost": (
        AdaBoostClassifier(random_state=42),
        {
            "n_estimators": [30, 50, 100],
            "learning_rate": [0.1, 0.5, 1.0],
            "algorithm": ["SAMME"]
        }
    ),
    "Naive Bayes": (
        GaussianNB(),
        {"var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6]}
    ),
    "QDA": (
        QuadraticDiscriminantAnalysis(),
        {"reg_param": [0, 0.01, 0.1, 0.5]}
    )
}

# Run BayesSearchCV for each model
results = {}
for name, (model_estimator, param_grid) in models_and_params.items():
    print(f"\n🔍 Training {name}...")

    # Use BayesSearchCV for hyperparameter optimization for each model
    opt = BayesSearchCV(
        estimator=model_estimator,  # Use the current model from the loop
        search_spaces=param_grid,   # Use the current param_grid from the loop
        cv=5,          # cross-validation
        n_jobs=-1,     # all cores
        n_iter=20,     # number of iterations for Bayesian optimization (reduced for faster demo)
        random_state=42,
        verbose=0,
        refit=True
    )

    # Fit on training data (using resampled data)
    opt.fit(x_train_res, y_train_res)

    y_pred = opt.predict(x_test)

    # Check if the estimator has predict_proba before calculating ROC AUC and PR AUC
    if hasattr(opt.best_estimator_, 'predict_proba'):
        y_prob = opt.predict_proba(x_test)[:, 1]
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
        print("ROC AUC:", roc_auc_score(y_test, y_prob))
        print("PR AUC (average precision):", average_precision_score(y_test, y_prob))
    else:
        print(f"Warning: {name} does not support predict_proba. Skipping ROC AUC and PR AUC.")
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
        print("ROC AUC: Not available")
        print("PR AUC (average precision): Not available")

    print("Best params:", opt.best_params_)
    print("Best CV score:", opt.best_score_)

    # Store results
    results[name] = {
        "best_params": opt.best_params_,
        "best_score": opt.best_score_,
        "test_score": opt.score(x_test, y_test)
    }


🔍 Training KNN...


/usr/local/lib/python3.12/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [20, 30] was inferred to Integer(low=20, high=30, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(20, 30), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [20, 30] was inferred to Integer(low=20, high=30, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(20, 30), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [20, 30] was inferred to Integer(low=20, high=30, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be infer

Accuracy: 0.8727272727272727
              precision    recall  f1-score   support

           0       0.90      0.87      0.88        30
           1       0.85      0.88      0.86        25

    accuracy                           0.87        55
   macro avg       0.87      0.87      0.87        55
weighted avg       0.87      0.87      0.87        55

Confusion matrix:
 [[26  4]
 [ 3 22]]
ROC AUC: 0.8653333333333333
PR AUC (average precision): 0.8042621905098686
Best params: OrderedDict({'model__leaf_size': 28, 'model__n_neighbors': 5, 'model__weights': 'distance'})
Best CV score: 0.777613320999075

🔍 Training SVM...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(10), np.str_('auto'), np.str_('rbf')] before, using random point [10, 'scale', 'rbf']
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1), np.str_('auto'), np.str_('rbf')] before, using random point [0.1, 'scale', 'rbf']
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(10), np.str_('auto'), np.str_('rbf')] before, using random point [0.1, 'scale', 'rbf']
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1), np.str_('scale'), np.str_('rbf')] before, using random point [10, 'auto', 'rbf']
  warnings.warn(
/usr/local/lib/python3.1

Accuracy: 0.8363636363636363
              precision    recall  f1-score   support

           0       0.86      0.83      0.85        30
           1       0.81      0.84      0.82        25

    accuracy                           0.84        55
   macro avg       0.83      0.84      0.84        55
weighted avg       0.84      0.84      0.84        55

Confusion matrix:
 [[25  5]
 [ 4 21]]
ROC AUC: 0.908
PR AUC (average precision): 0.8750673778798157
Best params: OrderedDict({'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'})
Best CV score: 0.7862164662349678

🔍 Training Gaussian Process...


...

Accuracy: 0.8727272727272727
              precision    recall  f1-score   support

           0       0.93      0.83      0.88        30
           1       0.82      0.92      0.87        25

    accuracy                           0.87        55
   macro avg       0.87      0.88      0.87        55
weighted avg       0.88      0.87      0.87        55

Confusion matrix:
 [[25  5]
 [ 2 23]]
ROC AUC: 0.9460000000000001
PR AUC (average precision): 0.9356216145584972
Best params: OrderedDict({'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 100})
Best CV score: 0.7689176688251618

🔍 Training Naive Bayes...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(1e-07)] before, using random point [1e-08]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(1e-07)] before, using random point [1e-07]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(1e-07)] before, using random point [1e-08]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(1e-09)] before, using random point [1e-09]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(1e-06)] before, using random point [1e-06]
  warnings.w

Accuracy: 0.7818181818181819
              precision    recall  f1-score   support

           0       0.72      0.97      0.83        30
           1       0.93      0.56      0.70        25

    accuracy                           0.78        55
   macro avg       0.83      0.76      0.76        55
weighted avg       0.82      0.78      0.77        55

Confusion matrix:
 [[29  1]
 [11 14]]
ROC AUC: 0.8739999999999999
PR AUC (average precision): 0.8562069371768111
Best params: OrderedDict({'var_smoothing': 1e-08})
Best CV score: 0.6961147086031452

🔍 Training QDA...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.1]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(0)] before, using random point [0]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.5)] before, using random point [0.5]
  warnings.warn(
/usr/local/lib/pyth

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.92      0.88      0.90        25

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55

Confusion matrix:
 [[28  2]
 [ 3 22]]
ROC AUC: 0.932
PR AUC (average precision): 0.9309052424738115
Best params: OrderedDict({'reg_param': 0.1})
Best CV score: 0.7516188714153562


In [10]:
!pip install catboost
from sklearn.ensemble import BaggingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier



bag_params = {
    "estimator": [DecisionTreeClassifier(), None],
    "n_estimators": [6, 8, 10, 12],
    "max_samples": [0.5, 0.7, 1.0],
    "bootstrap": [True, False],
    "bootstrap_features": [True, False],
    "oob_score": [True, False]
}


from sklearn.linear_model import LogisticRegression

stack_params = {
    "stack_method": ["auto", "predict_proba", "predict"], # Removed 'decision_function'
    "passthrough": [True, False],
    "final_estimator": [
        LogisticRegression(),
        CatBoostClassifier(verbose=0)
    ]
}

cat_params={
     "iterations": [20, 30, 40],
    "learning_rate": [0.1, 0.01, 0.5],
    "depth": [5, 6, 7]
}

params={
    'bagging':bag_params,
    'stacking':stack_params,
    'catboost':cat_params}

models_and_params = {
    "bagging": (
        BaggingClassifier(random_state=42),
        bag_params
    ),

    "stacking": (
        StackingClassifier(
            estimators=[
                ("svc", SVC(probability=True)),
                ("dt", DecisionTreeClassifier())
            ],
            final_estimator=LogisticRegression()
        ),
        stack_params
    ),

    "catboost": (
        CatBoostClassifier(verbose=0),
        cat_params
    )
}



for name, (model, params) in models_and_params.items():
    print(f"\n Training {name}...")


    opt.fit(x_train_res, y_train_res)
    y_pred = opt.predict(x_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_prob))
    print("PR AUC (average precision):", average_precision_score(y_test, y_prob))


    print("Best params:", opt.best_params_)
    print("Best CV score:", opt.best_score_)

    # Store results if needed
    results[name] = {
        "best_params": opt.best_params_,
        "best_score": opt.best_score_,
        "test_score": opt.score(x_test, y_test)
    }

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00

 Training bagging...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.1]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(0)] before, using random point [0]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.5)] before, using random point [0.5]
  warnings.warn(
/usr/local/lib/pyth

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.92      0.88      0.90        25

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55

Confusion matrix:
 [[28  2]
 [ 3 22]]
ROC AUC: 0.932
PR AUC (average precision): 0.9309052424738115
Best params: OrderedDict({'reg_param': 0.1})
Best CV score: 0.7516188714153562

 Training stacking...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.1]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(0)] before, using random point [0]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.5)] before, using random point [0.5]
  warnings.warn(
/usr/local/lib/pyth

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.92      0.88      0.90        25

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55

Confusion matrix:
 [[28  2]
 [ 3 22]]
ROC AUC: 0.932
PR AUC (average precision): 0.9309052424738115
Best params: OrderedDict({'reg_param': 0.1})
Best CV score: 0.7516188714153562

 Training catboost...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.1]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(0)] before, using random point [0]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.5)] before, using random point [0.5]
  warnings.warn(
/usr/local/lib/pyth

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.92      0.88      0.90        25

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55

Confusion matrix:
 [[28  2]
 [ 3 22]]
ROC AUC: 0.932
PR AUC (average precision): 0.9309052424738115
Best params: OrderedDict({'reg_param': 0.1})
Best CV score: 0.7516188714153562


In [12]:
stack_params = {
    "stack_method": ["auto", "predict_proba", "predict"], # Removed 'decision_function'
    "passthrough": [True, False],
    "final_estimator": [
        LogisticRegression(),
        CatBoostClassifier(verbose=0)
    ]
}

params={'stacking':stack_params}

models_and_params = {


    "stacking": (
        StackingClassifier(
            estimators=[
                ("svc", SVC(probability=True)),
                ("RF", RandomForestClassifier(random_state=42))
            ],
            final_estimator=LogisticRegression()
        ),
        stack_params
    )
}
for name, (model, params) in models_and_params.items():
    print(f"\n Training {name}...")


    opt.fit(x_train_res, y_train_res)
    y_pred = opt.predict(x_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_prob))
    print("PR AUC (average precision):", average_precision_score(y_test, y_prob))


    print("Best params:", opt.best_params_)
    print("Best CV score:", opt.best_score_)

    # Store results if needed
    results[name] = {
        "best_params": opt.best_params_,
        "best_score": opt.best_score_,
        "test_score": opt.score(x_test, y_test)
    }




 Training stacking...


/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.1]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.1)] before, using random point [0.01]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(0)] before, using random point [0]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.float64(0.5)] before, using random point [0.5]
  warnings.warn(
/usr/local/lib/pyth

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.92      0.88      0.90        25

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55

Confusion matrix:
 [[28  2]
 [ 3 22]]
ROC AUC: 0.932
PR AUC (average precision): 0.9309052424738115
Best params: OrderedDict({'reg_param': 0.1})
Best CV score: 0.7516188714153562
